<a href="https://colab.research.google.com/github/adepriciliaa/Expert-System/blob/main/Final_Test_Expert_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import library
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn import model_selection, preprocessing, linear_model, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import itertools

In [ ]:
#Read Dataset
from google.colab import drive
drive.mount('/content/drive')
#import data dari Gdrive
data = pd.read_csv("/content/drive/MyDrive/ExpertSystem/consumer_complaints.csv",encoding='latin-1')
#membaca data yang diinput
data.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [ ]:
#Mengambil data kolo 'Product' dan  'Consumer_complaint_narrative
data = complaint[['product', 'consumer_complaint_narrative']]
#Menghilangkan data yang NULL
data = data[pd.notnull(data['consumer_complaint_narrative'])]
#Menampilkan 5 data dari bawah dengan menggunakan tail
data.tail(5)

,product,consumer_complaint_narrative
553084,Credit reporting,XXXX XXXX is reporting incorrectly payments ha...
553085,Credit reporting,Reflecting incorrect payment status Have been ...
553086,Payday loan,I have been paying 18000 a month through direc...
553090,Mortgage,I recently became aware that Amerisave Mortgag...
553096,Mortgage,Bank of America has demonstrated an ongoing le...


In [ ]:
#lowercase untuk data kolom ‘consumer_complaint_narrative’
#menjadikan text pada data menjadi huruf kecil semua
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].apply(lambda x: " ".join(x.lower() for x in x.split()))
print(data.consumer_complaint_narrative.tail(5))

553084    xxxx xxxx is reporting incorrectly payments ha...
553085    reflecting incorrect payment status have been ...
553086    i have been paying 18000 a month through direc...
553090    i recently became aware that amerisave mortgag...
553096    bank of america has demonstrated an ongoing le...
Name: consumer_complaint_narrative, dtype: object


In [ ]:
#remove puntuaction: symbol
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].str.replace('[^\w\s]', '')
#menampilkan data yang sudah di upgrade
print(data.consumer_complaint_narrative.tail(5))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


553084    xxxx xxxx is reporting incorrectly payments ha...
553085    reflecting incorrect payment status have been ...
553086    i have been paying 18000 a month through direc...
553090    i recently became aware that amerisave mortgag...
553096    bank of america has demonstrated an ongoing le...
Name: consumer_complaint_narrative, dtype: object


In [ ]:
#remove stopwords : kata-kata yang sering muncul dan tidak memiliki makna/arti
#kata yang digunakan bahasa english
stop = stopwords.words('english')
data['consumer_complaint_narrative'] = data['consumer_complaint_narrative'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))
#menampilkan data
print(data.consumer_complaint_narrative.tail(5))

553084    xxxx xxxx reporting incorrectly payments time ...
553085             reflecting incorrect payment status time
553086    paying 18000 month direct debit withdrawal che...
553090    recently became aware amerisave mortgage corpo...
553096    bank america demonstrated ongoing level incomp...
Name: consumer_complaint_narrative, dtype: object


In [ ]:
#membagi data menjadi 2 yaitu train dan valid
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data['consumer_complaint_narrative'], data['product'], test_size = 0.05)
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#membuat model menggunakan tfidf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, ngram_range=(1,1))

xtrain_tfidf =  tfidf_vect.fit_transform(train_x)
xvalid_tfidf =  tfidf_vect.fit_transform(valid_x)

tfidf_vect.fit(data['consumer_complaint_narrative'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

Model = linear_model.LogisticRegression().fit(xtrain_tfidf, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
                   penalty='12', random_state=None, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

accuracy = metrics.accuracy_score(Model.predict(xvalid_tfidf), valid_y)
print('Accuracy: ', accuracy)

print(metrics.classification_report(valid_y, Model.predict(xvalid_tfidf), target_names = data['product'].unique()))

Accuracy:  0.8581263094881771
                         precision    recall  f1-score   support

        Debt collection       0.85      0.82      0.83       280
          Consumer Loan       0.78      0.60      0.68       181
               Mortgage       0.81      0.83      0.82       409
            Credit card       0.85      0.86      0.86       644
       Credit reporting       0.83      0.90      0.86       871
           Student loan       0.91      0.75      0.82        28
Bank account or service       0.94      0.96      0.95       733
            Payday loan       0.00      0.00      0.00         7
        Money transfers       0.62      0.24      0.34        34
Other financial service       0.82      0.71      0.76        52
           Prepaid card       0.88      0.81      0.85       102

               accuracy                           0.86      3341
              macro avg       0.75      0.68      0.71      3341
           weighted avg       0.85      0.86      0.85    

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

In [ ]:
#membuat model dengan menggunakan count vectorizer
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)

xtrain_count =  count_vect.fit_transform(train_x)
xvalid_count =  count_vect.fit_transform(valid_x)

count_vect.fit(data['consumer_complaint_narrative'])
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

model = linear_model.LogisticRegression().fit(xtrain_count, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
                   penalty='12', random_state=None, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

accuracy = metrics.accuracy_score(model.predict(xvalid_count), valid_y)
print('Accuracy: ', accuracy)

print(metrics.classification_report(valid_y, model.predict(xvalid_count), target_names = data['product'].unique()))

Accuracy:  0.8356779407363065
                         precision    recall  f1-score   support

        Debt collection       0.80      0.75      0.77       280
          Consumer Loan       0.70      0.56      0.62       181
               Mortgage       0.82      0.82      0.82       409
            Credit card       0.83      0.85      0.84       644
       Credit reporting       0.82      0.87      0.84       871
           Student loan       0.72      0.75      0.74        28
Bank account or service       0.93      0.94      0.93       733
            Payday loan       0.00      0.00      0.00         7
        Money transfers       0.43      0.26      0.33        34
Other financial service       0.77      0.71      0.74        52
           Prepaid card       0.85      0.79      0.82       102

               accuracy                           0.84      3341
              macro avg       0.70      0.66      0.68      3341
           weighted avg       0.83      0.84      0.83    

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
